The objective is to solve the exercises in https://docs.google.com/presentation/d/1T1nXunoeTV05A6itZhvf4NbhbQUoteOQoAxaa2aPlsw/edit#slide=id.g27e9614b9c_103_0.

I am using the House Price Prediction from https://www.kaggle.com/datasets/kirbysasuke/house-price-prediction-simplified-for-regression. The dataset should be downloaded from this source and sotred in the data folder.

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

<h1>Data transformation and preparation</h1>

In [2]:
df_input = pd.read_csv('./data/Real_Estate.csv')

In [3]:
df_input

,Transaction date,House age,Distance to the nearest MRT station,Number of convenience stores,Latitude,Longitude,House price of unit area
0,2012-09-02 16:42:30.519336,13.3,4082.01500,8,25.007059,121.561694,6.488673
1,2012-09-04 22:52:29.919544,35.5,274.01440,2,25.012148,121.546990,24.970725
2,2012-09-05 01:10:52.349449,1.1,1978.67100,10,25.003850,121.528336,26.694267
3,2012-09-05 13:26:01.189083,22.2,1055.06700,5,24.962887,121.482178,38.091638
4,2012-09-06 08:29:47.910523,8.5,967.40000,6,25.011037,121.479946,21.654710
...,...,...,...,...,...,...,...
409,2013-07-25 15:30:36.565239,18.3,170.12890,6,24.981186,121.486798,29.096310
410,2013-07-26 17:16:34.019780,11.9,323.69120,2,24.950070,121.483918,33.871347
411,2013-07-28 21:47:23.339050,0.0,451.64190,8,24.963901,121.543387,25.255105
412,2013-07-29 13:33:29.405317,35.9,292.99780,5,24.997863,121.558286,25.285620


In [4]:
df_input.describe()

,House age,Distance to the nearest MRT station,Number of convenience stores,Latitude,Longitude,House price of unit area
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,18.405072,1064.468233,4.265700,24.973605,121.520268,29.102149
std,11.757670,1196.749385,2.880498,0.024178,0.026989,15.750935
min,0.000000,23.382840,0.000000,24.932075,121.473888,0.000000
25%,9.900000,289.324800,2.000000,24.952422,121.496866,18.422493
50%,16.450000,506.114400,5.000000,24.974353,121.520912,30.394070
75%,30.375000,1454.279000,6.750000,24.994947,121.544676,40.615184
max,42.700000,6306.153000,10.000000,25.014578,121.565321,65.571716


In [5]:
df_input.dtypes

Transaction date                        object
House age                              float64
Distance to the nearest MRT station    float64
Number of convenience stores             int64
Latitude                               float64
Longitude                              float64
House price of unit area               float64
dtype: object

Translating transaction date to unix timestamp

In [6]:
df_input['Transaction date'] = pd.to_datetime(df_input['Transaction date']).astype('int64') // 10**9
df_input

,Transaction date,House age,Distance to the nearest MRT station,Number of convenience stores,Latitude,Longitude,House price of unit area
0,1346604150,13.3,4082.01500,8,25.007059,121.561694,6.488673
1,1346799149,35.5,274.01440,2,25.012148,121.546990,24.970725
2,1346807452,1.1,1978.67100,10,25.003850,121.528336,26.694267
3,1346851561,22.2,1055.06700,5,24.962887,121.482178,38.091638
4,1346920187,8.5,967.40000,6,25.011037,121.479946,21.654710
...,...,...,...,...,...,...,...
409,1374766236,18.3,170.12890,6,24.981186,121.486798,29.096310
410,1374858994,11.9,323.69120,2,24.950070,121.483918,33.871347
411,1375048043,0.0,451.64190,8,24.963901,121.543387,25.255105
412,1375104809,35.9,292.99780,5,24.997863,121.558286,25.285620


Standardise input features

In [7]:
df_x = df_input.drop('House price of unit area', axis = 1)
df_x = (df_x - df_x.mean(axis = 0)) / df_x.std(axis = 0)
df_x.describe()

,Transaction date,House age,Distance to the nearest MRT station,Number of convenience stores,Latitude,Longitude
count,4.140000e+02,4.140000e+02,4.140000e+02,4.140000e+02,4.140000e+02,4.140000e+02
mean,-1.139614e-14,-1.802101e-16,5.148860e-17,-9.439577e-17,7.311167e-14,-6.866863e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.680271e+00,-1.565367e+00,-8.699277e-01,-1.480890e+00,-1.717652e+00,-1.718473e+00
25%,-8.633026e-01,-7.233638e-01,-6.477074e-01,-7.865654e-01,-8.761327e-01,-8.670862e-01
50%,-1.133826e-02,-1.662806e-01,-4.665587e-01,2.549210e-01,3.094246e-02,2.385681e-02
75%,8.047446e-01,1.018053e+00,3.257246e-01,8.624547e-01,8.826960e-01,9.043639e-01
max,1.740240e+00,2.066305e+00,4.379935e+00,1.990732e+00,1.694608e+00,1.669315e+00


Train/test split

In [8]:
X = df_x.values
y = df_input['House price of unit area'].values

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=111)

Creating data wrappers for PyTorch

In [9]:
train_set = TensorDataset(torch.from_numpy(X_train).unsqueeze(1), torch.from_numpy(y_train))
test_set = TensorDataset(torch.from_numpy(X_test).unsqueeze(1), torch.from_numpy(y_test))

In [10]:
for i in range(3):
    sample_idx = torch.randint(len(train_set), size=(1,)).item()
    x, y = train_set[sample_idx]
    print('x = ' + str(x))
    print('y = ' + str(y))
    print()

x = tensor([[ 1.0500, -0.0770,  0.4693,  0.2549, -1.5840,  1.5753]],
       dtype=torch.float64)
y = tensor(21.2229, dtype=torch.float64)

x = tensor([[-0.1889, -0.3832, -0.6137, -1.4809, -0.5069,  0.8499]],
       dtype=torch.float64)
y = tensor(13.8489, dtype=torch.float64)

x = tensor([[-1.1049,  0.5864, -0.4065,  0.2549,  0.5092,  0.4867]],
       dtype=torch.float64)
y = tensor(20.1690, dtype=torch.float64)



<h1>Defining a linear regression NN in PyTorch</h1>